In [ ]:
from google.colab import drive
drive.mount('content')

df = pd.read_csv('/content/content/MyDrive/data/wanted.csv')
year_id_df = pd.read_csv('/content/year_id_df.csv')

df_engined = engineering(df)
df = year_mapping(year_id_df,df_engined)

## DB 테이블 생성 및 데이터 삽입

In [ ]:
import psycopg2

host = 'arjuna.db.elephantsql.com'
user = 'pvltcvea'
password = 'koXbh3YWTCItiJduk1ioNrRBp3eOhlbz'
database = 'pvltcvea'

conn = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
cur = conn.cursor()

cur.execute("DROP TABLE IF EXISTS job;")
cur.execute("""
    CREATE TABLE IF NOT EXISTS job(
        id INTEGER NOT NULL PRIMARY KEY,
        skill_tags VARCHAR(1000),
        category_tags VARCHAR(1000),
        requirements VARCHAR(10000),
        main_tasks VARCHAR(10000),
        preferred_points VARCHAR(10000),
        company VARCHAR(1000),
        logo_img VARCHAR(1000),
        url VARCHAR(1000),
        career VARCHAR(100)
    );
""")

In [ ]:
for i in range(len(df)):
  id = df.loc[i,'id']
  skill_tags = ' '.join(df.loc[i,'skill_tags'])
  category_tags = ', '.join(df.loc[i,'category_tags'])
  requirements = df.loc[i,'detail.requirements']
  main_tasks = df.loc[i,'detail.main_tasks']
  preferred_points = df.loc[i,'detail.preferred_points']
  name = df.loc[i,'company.name']
  logo_img = df.loc[i,'logo_img.origin']
  url = df.loc[i,'url']
  career = ''
  if df.loc[i,'year'][0] == -1 and df.loc[i,'year'][1] == -1:
    career = '-1'
  elif df.loc[i,'year'][0] == -1:
    for num in range(0, df.loc[i,'year'][1]+1):
      if num > 9:
        break
      career += str(num)
  elif df.loc[i,'year'][1] == -1:
    for num in range(df.loc[i,'year'][0], 10):
      if num > 9:
        break
      career += str(num)
  else:
    for num in range(df.loc[i,'year'][0], df.loc[i,'year'][1]+1):
      if num > 9:
        break
      career += str(num)
  cur.execute("""
    INSERT INTO job (id,skill_tags,category_tags,requirements,main_tasks,preferred_points,company,logo_img,url,career) VALUES (%s,%s,%s
    ,%s,%s,%s,%s,%s,%s,%s);
  """, (int(id),skill_tags,category_tags,requirements,main_tasks,preferred_points,name,logo_img,url,career))

conn.commit()

cur.close()
conn.close()